# [Chapter 2. Strings and Text](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html)

Almost every useful program involves some kind of processing, whether it is parsing data or generating output.  
This chapter focuses on common problems involving text manipulation, such as pulling apart strings, searching, substitution, lexing, and parsing.  
Many of these tasks can be easily solved using built-in methods of strings.  
However, more complicated operations might require the use of regular expressions or the creation of a full-fledged parser.  
All of those topics will be covered.  
In addition, a few tricky aspects of [working with Unicode](https://en.wikipedia.org/wiki/Unicode) are addressed.  
Let's do this!

## [Splitting Strings on Any of Multiple Delimiters](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html#_splitting_strings_on_any_of_multiple_delimiters)

### Problem 

You need to split a string into fields, but the delimiters (and spacing around them) aren't consistent throughout the string.

### Solution

The `split()` method of string objects is really meant for very simple cases, and does not allow for multiple delimiters or account for possible whitespace around the delimiters.  
In cases when you need a bit more flexibility, use the [re.split() method](https://docs.python.org/3/library/re.html#re.split):

In [72]:
import re

line = 'asdf fjdk; afed, fjek,asdf,      foo'
re.split(r'[;,\s]\s*', line)

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']

Here's some [examples from the documentation](https://docs.python.org/3/library/re.html#re.split):

In [73]:
re.split('\W+', 'Words, words, words.')

['Words', 'words', 'words', '']

In [74]:
re.split('(\W+)', 'Words, words, words.')

['Words', ', ', 'words', ', ', 'words', '.', '']

In [75]:
re.split('\W+', 'Words, words, words.', 1)

['Words', 'words, words.']

In [76]:
re.split('[a-f]+', '0a3B9', flags=re.IGNORECASE)

['0', '3', '9']

### Discussion

The `re.split()` function is useful because you can specify multiple patterns for the separator.  
For example, as shown in the solution, the separator is either a comma (,), semicolon (;), or whitespace followed by any amount of extra whitespace.  
Whenever that pattern is found, the entire match becomes the delimiter between whatever fields lie on either side of the match.  
The result is a list of fields, just as with `str.split()`.

When using `re.split()`, you need to be a bit careful should the regular expression pattern involve a capture group enclosed in parentheses.  
If capture groups are used, then the matched text is also included in the result.  
For example, watch what happens here:

In [77]:
line = 'asdf fjdk; afed, fjek,asdf,      foo'
fields = re.split(r'(;|,|\s)\s*', line)
fields

['asdf', ' ', 'fjdk', ';', 'afed', ',', 'fjek', ',', 'asdf', ',', 'foo']

Getting the split characters might be useful in certain contexts.  
For example, maybe you need the split characters later on to reform an output string:

In [78]:
fields

['asdf', ' ', 'fjdk', ';', 'afed', ',', 'fjek', ',', 'asdf', ',', 'foo']

In [79]:
values = fields[::2]
values

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']

In [80]:
delimiters = fields[1::2] + ['']
delimiters

[' ', ';', ',', ',', ',', '']

In [81]:
# Now re-form the line using the same delimiters and values:
''.join(v+d for v, d in zip(values, delimiters))

'asdf fjdk;afed,fjek,asdf,foo'

In [82]:
# The original line for reference:
line

'asdf fjdk; afed, fjek,asdf,      foo'

If you don't want the separator characters in the result, but still need to use parentheses to group parts of the regular expression pattern, make sure you use a [noncapture group](https://stackoverflow.com/questions/3512471/what-is-a-non-capturing-group-what-does-a-question-mark-followed-by-a-colon), specified as `(?:...)`.

In [83]:
re.split(r'(?:,|;|\s)\s*', line)

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']

## [Matching Text at the Start or End of a String](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html#_matching_text_at_the_start_or_end_of_a_string)

### Problem

You need to check the start or end of a string for specific text patterns, such as filename extensions, URL schemes, and so on.

### Solution

A simple way to check the beginning or end of a string is to use the `str.startswith()` or `str.endswith()` methods.

In [84]:
filename = 'spam.txt'
filename.endswith('.txt')

True

In [85]:
filename.startswith('file:')

False

In [86]:
url = 'http://www.python.org'
url.startswith('http:')

True

If you need to check against multiple choices, simply provide a tuple of possibilities to `startswith()` or `endswith()`:

In [87]:
import os

filenames = os.listdir('.')
filenames

['.ipynb_checkpoints',
 '1_Chapter.ipynb',
 '2_Chapter.ipynb',
 'python_ipsum.txt']

In [88]:
[name for name in filenames if name.endswith(('.ipynb', '.txt'))]

['1_Chapter.ipynb', '2_Chapter.ipynb', 'python_ipsum.txt']

In [89]:
any(name.endswith('.ipynb') for name in filenames)

True

Another example, for your edutainment:

In [90]:
from urllib.request import urlopen

def read_data(name):
    if name.startswith(('http:', 'https:', 'ftp:')):
        return urlopen(name).read()
    else:
        with open(name) as f:
            return f.read()
    f.close()

Oddly, this is one part of Python where a tuple is actually required as input.  
If you happen to have the choices specified in a list or set, just make sure that you convert them to a tuple  (using `tuple()`) first.

In [91]:
choices = ['http:', 'ftp']
url = 'http://www.python.org'

In [92]:
url.startswith(tuple(choices))

True

### Discussion

The `startswith()` and `endswith()` methods provide a very convenient way to perform basic prefix and suffix checking.  
Similar operations can be performed with slices, but are far less elegant.

In [93]:
filename = 'spam.txt'
filename[-4:] == '.txt'

True

In [94]:
url = 'http://www.python.org'
url[:5] == 'http:' or url[:6] == 'https:' or url[:4] == 'ftp:'

True

You might also be inclined to use regular expressions as an alternative.

In [95]:
import re

url = 'http://www.python.org'
re.match('http:|https:|ftp:', url)

<_sre.SRE_Match object; span=(0, 5), match='http:'>

This works, but it is often overkill for simple matching.  
Using the str.startswith() or str.endswith() methods is simpler and much faster.

Last, but not least, the `startswith()` and `endswith()` methods look nice when combined with other operations, such as common data reductions.  
In the next example, this statement checks a directory for the presence of certain kinds of files:

## [Matching Strings Using Shell Wildcard Patterns](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html#_matching_strings_using_shell_wildcard_patterns)

### Problem

You want to match text using the same wildcard patterns that are commonly used when working with Unix shells (e.g., `*.py, Dat[0-9]*.csv`, and so on).

### Solution

The `fnmatch` module provides two functions -- `fnmatch()` and `fnmatchcase()` -- that can be used to perform such matching.

In [96]:
# Here's an example from the Python 3 documentation:

import fnmatch
import os

for file in os.listdir('.'):
    if fnmatch.fnmatch(file, '*.txt'):
        print(file)

python_ipsum.txt


In [97]:
from fnmatch import fnmatch, fnmatchcase

In [98]:
fnmatch('foo.txt', '*.txt')

True

In [99]:
fnmatch('foo.txt', '?oo.txt')

True

In [100]:
fnmatch('Dat45.csv', 'Dat[0-9]*')

True

In [101]:
names = ['Dat1.csv', 'Dat2.csv', 'config.ini', 'foo.py']
[name for name in names if fnmatch(name, 'Dat*.csv')]

['Dat1.csv', 'Dat2.csv']

Normally, `fnmatch()` matches patterns using the same case-sensitivity rules as the system's undelying filesystem (which varies based on the operating system).

In [102]:
# On Mac OSX:
fnmatch('foo.txt', '*.TXT')

False

If this distinction matters, use `fnmatchcase()` instead.  
It matches exactly based on the lower- and upper-case conventions that you supply:

In [103]:
fnmatchcase('foo.txt', '*.TXT')

False

An often overlooked feature of these functions is their potential use with data processing of nonfilename strings.  
For example, suppose that you have a list of street addresses like this:

In [104]:
addresses = [
    '5412 N CLARK ST',
    '1060 W ADDISON ST',
    '1039 W GRANVILLE AVE',
    '2122 N CLARK ST',
    '4802 N BROADWAY',
]

You could write list comprehensions like this:

In [105]:
from fnmatch import fnmatchcase

In [106]:
[addr for addr in addresses if fnmatchcase(addr, '* ST')]

['5412 N CLARK ST', '1060 W ADDISON ST', '2122 N CLARK ST']

In [107]:
[addr for addr in addresses if fnmatchcase(addr, '54[0-9][0-9] *CLARK*')]

['5412 N CLARK ST']

### Discussion

The matching performed by `fnmatch` sits somewhere between the functionality of simple string methods and the full power of regular expressions.  
If you're just trying to provide a simple mechanism for allowing wildcards in data processing operations, it's often a reasonable solution.  
If you're actually trying to write code that matches filenames, use the `glob` module instead.  
See ["Getting A Directory Listing"](http://chimera.labs.oreilly.com/books/1230000000393/ch05.html#dirlisting).

## [Matching and Searching for Text Patterns](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html#_matching_and_searching_for_text_patterns)